In [6]:
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from google.cloud import storage
#from torch.utils.tensorboard import SummaryWriter
# %load_ext tensorboard
import datetime
import time
import torch.nn as nn

from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck

#torch.cuda.empty_cache() 

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [2]:
def process_img(image):
    pil_to_tensor = transforms.ToTensor()
    resize = transforms.Resize((225,225))
    img = Image.open(image).convert('RGB')
    
    features = pil_to_tensor(img)
    features = resize(features)
    features = torch.unsqueeze(features, dim=0)
    print(features.shape)
    return features

In [3]:
## Created a classifier based on the RESNET50 pretrained model

class ItemFeatureExtractor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        #self.resnet50 = model
        self.resnet50.fc = torch.nn.Linear(2048,13)
  
    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [4]:
model = ItemFeatureExtractor()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
checkpoint = torch.load('model_test/weights.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']


Using cache found in /home/jupyter/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [7]:
## remove last layer to 2048 feature output
# Remove the last layer
model = nn.Sequential(*list(model.resnet50.children())[:-1])

In [8]:
# Pass the input tensor through the model
model.eval()  # Set the model to evaluation mode
output = model(process_img('pytorch_images/appliances/018.jpeg'))

torch.Size([1, 3, 225, 225])


In [10]:
## Test that 2048 features are being extracted
output.shape

torch.Size([1, 2048, 1, 1])